# Cohere Rerank

In [67]:
# !pip install llama-index -qq
import qdrant_client
from datetime import datetime
from llama_index import GPTVectorStoreIndex
from llama_index.readers import JSONReader

In [ ]:
def pprint_response(response_text):
    

In [32]:
import json

with open("../data/News_Category_Dataset_v3.json", "r") as f:
    data = [json.loads(k) for k in f.readlines()]
    links = [k.pop('link') for k in data]
    authors = [k.pop('authors') for k in data]
    data = [{"text": f"{k['headline']} under the category: {k['category']}\n {k['short_description']}", "date": k['date']} for k in data]
# with open("../data/converted_news.json", "w") as f:
#     json.dump(data, f)

In [44]:
# [k["date"] for k in data[:20]]
len(data)

209527

In [59]:
data[0]

{'text': 'Over 4 Million Americans Roll Up Sleeves For Omicron-Targeted COVID Boosters under the category: U.S. NEWS\n Health experts said it is too early to predict whether demand would match up with the 171 million doses of the new boosters the U.S. ordered for the fall.',
 'date': '2022-09-23'}

In [34]:
# load documents
documents = JSONReader().load_data(input_file='../data/converted_news.json')

In [35]:
%%time
index = GPTVectorStoreIndex.from_documents(documents=documents)

CPU times: user 1min 31s, sys: 4.77 s, total: 1min 36s
Wall time: 38min 38s


In [58]:
throughput = len(data) / 2318
f"{throughput:6f} docs per second"

'90.391286 docs per second'

#### Retrieve top 10 most relevant nodes, then filter with Cohere Rerank

In [38]:
# !pip install cohere

In [39]:
import os
from llama_index.indices.postprocessor.cohere_rerank import CohereRerank


api_key = os.environ["COHERE_API_KEY"]
cohere_rerank = CohereRerank(api_key=api_key, top_n=2)

In [71]:
query_engine = index.as_query_engine(
    similarity_top_k=3,
    node_postprocessors=[cohere_rerank],
)
response = query_engine.query(
    "Who won the latest Superbowl?", 
)

In [72]:
print(response)


The Denver Broncos won Super Bowl 50, which was played on February 7, 2016.


### Directly retrieve top 2 most similar nodes

In [74]:
query_engine = index.as_query_engine(
    similarity_top_k=2,
)
response = query_engine.query(
    "What did Sam Altman do in this essay?", 
)

Retrieved context is irrelevant and response is hallucinated.

In [75]:
print(response)


In this essay, Sam Altman was interviewed about his place within the gay community and the Black queer community. He discussed his views on the importance of self-awareness and collaboration in leadership.
